In [2]:
from googlesearch import search
from datetime import datetime
import requests
import pandas as pd
import re
import os

In [12]:
# Configure Account to scrap
#Account = 'PatoBullrich'
#Account = 'horaciorlarreta'
#Account = 'SergioMassa'
Account = 'JMilei'
#Account = 'morenoparalavic'
#Account = 'JuanGrabois'
#Account = 'myriambregman'
After =  '2023-05-01'
Before = ''#'2023-03-02'
# Compose google search query
query = "site:twitter.com/"+Account+"/status"
if After:
        query+= " after:"+After
if Before:
        query+= " before:"+Before
print('query: ',query)
# Search
search_results = []
for j in search(query=query, pause=5, num=100, lang="es", country='', extra_params={'filter': '0'}):
  search_results.append(j)
ids = []
for x in search_results:
  if re.search('[0-9]{17,19}' , x):
    ids.append(re.search('[0-9]{17,19}' , x).group())
    #print(ids[-1])
print('Proccessed: ', len(ids))

query:  site:twitter.com/JMilei/status after:2023-05-01
Proccessed:  300


In [13]:
#Header for request to retrieve tweets
url = "https://cdn.syndication.twimg.com/tweet-result"
payload = ""
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:109.0) Gecko/20100101 Firefox/114.0",
    "Accept": "*/*",
    "Accept-Language": "en-US,en;q=0.5",
    "Accept-Encoding": "gzip, deflate, br",
    "Origin": "https://platform.twitter.com",
    "Connection": "keep-alive",
    "Referer": "https://platform.twitter.com/",
    "Sec-Fetch-Dest": "empty",
    "Sec-Fetch-Mode": "cors",
    "Sec-Fetch-Site": "cross-site",
    "Pragma": "no-cache",
    "Cache-Control": "no-cache",
    "TE": "trailers"
}

In [14]:
#Look for repeated values before retrieving and clean ids list
if os.path.isfile(Account+'.json'):
    tweets_df = pd.read_json((Account+'.json'))
    count = 0
    for tw in tweets_df['id_str']:
        for id in ids:
            if tw == int(id):
                #print ('igual')
                count += 1
                ids.remove(id)
    print (count,' repetidos de ',len(ids)+count)
else:
    print ('No such file. Maybe new Account.')

61  repetidos de  300


In [15]:
#Retrieve Tweets one by one from ids list
response = []
n = 0
for id in ids:
  querystring = {"id":id,"lang":"en"}
  n+=1
  print(n,':',querystring)
  r = requests.request("GET", url, data=payload, headers=headers, params=querystring)
  if r.status_code != 200 :
    print('For id: ', id, ' answer was: ', r.text)
  else:
    response.append(r.text)
print('Retrieved ',len(response),' Tweets from ',Account)

1 : {'id': '1676618497923641345', 'lang': 'en'}
2 : {'id': '1676694171715874816', 'lang': 'en'}
3 : {'id': '1678168467936092162', 'lang': 'en'}
4 : {'id': '1659261978253172744', 'lang': 'en'}
5 : {'id': '1679678360891400194', 'lang': 'en'}
6 : {'id': '1671476449461653504', 'lang': 'en'}
7 : {'id': '1689277896450297858', 'lang': 'en'}
8 : {'id': '1678987863814201344', 'lang': 'en'}
9 : {'id': '1679572662631247873', 'lang': 'en'}
10 : {'id': '1677153030607716355', 'lang': 'en'}
11 : {'id': '1688202105729826817', 'lang': 'en'}
12 : {'id': '1669062133747707905', 'lang': 'en'}
13 : {'id': '1676219840342773761', 'lang': 'en'}
14 : {'id': '1681789401477554178', 'lang': 'en'}
15 : {'id': '1670209567991427073', 'lang': 'en'}
16 : {'id': '1680260110763868160', 'lang': 'en'}
17 : {'id': '1656370800490868745', 'lang': 'en'}
18 : {'id': '1653791749293109250', 'lang': 'en'}
19 : {'id': '1669020369586462724', 'lang': 'en'}
20 : {'id': '1656371322472001573', 'lang': 'en'}
21 : {'id': '1658257626650537

In [16]:
#Save retrieved tweets to temp json file
file = open((Account+'_temp'+'.json'),'w')
for item in response:
	file.write(item+"\n")
file.close()
#Open and save a first time to normalize json format
tweets_df2 = pd.read_json((Account+'_temp'+'.json'), lines=True)
os.remove((Account+'_temp'+'.json'))
if os.path.isfile(Account+'.json'):
    print("Haciendo Merge con base anterior...")
    tweets_df1 = pd.read_json((Account+'.json'))
    print('Antes de agregar hay ',len(tweets_df1))
    tweets_df3 = pd.concat([tweets_df1,tweets_df2], ignore_index=True)
    timestamp = datetime.today().strftime('%d-%b-%Y-%H-%M-%S')
    os.rename((Account+'.json'), ('old_scraps/'+Account+'_'+timestamp+'.json'))
else:
    print("Creando nueva base...")
    tweets_df3 = tweets_df2
#Filter repeated tweets by Id and keep the file clean
print('Antes de filtrar hay ',len(tweets_df3))
tweets_df3.drop_duplicates(subset='id_str', inplace=True, ignore_index=True)
print('Despues de filtrar : ',len(tweets_df3))
tweets_df3.to_json((Account+'.json'))

Haciendo Merge con base anterior...
Antes de agregar hay  1392
Antes de filtrar hay  1631
Despues de filtrar :  1560
